In [10]:
# -*- coding: utf-8 -*-

"""
将参考图像坐标系添加到目标图像上
~~~~~~~~~~~~~~~~
code by wHy
Aerospace Information Research Institute, Chinese Academy of Sciences
Ghent University
Haoyu.Wang@ugent.be
"""

from tqdm import tqdm
import gdal
import ogr
import fnmatch
from PIL import Image
from numpy import *
from pylab import *
from scipy.ndimage import filters
import cv2
import matplotlib.pyplot as plt
import os
import sys

In [11]:
def write_img(out_path, im_proj, im_geotrans, im_data):
    """output img

    Args:
        out_path: Output path
        im_proj: Affine transformation parameters
        im_geotrans: spatial reference
        im_data: Output image data

    """
    # identify data type 
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32

    # calculate number of bands
    if len(im_data.shape) > 2:  
        im_bands, im_height, im_width = im_data.shape
    else:  
        im_bands, (im_height, im_width) = 1, im_data.shape

    # create new img
    driver = gdal.GetDriverByName("GTiff")
    new_dataset = driver.Create(
        out_path, im_width, im_height, im_bands, datatype)
    new_dataset.SetGeoTransform(im_geotrans)
    new_dataset.SetProjection(im_proj)
    if im_bands == 1:
        new_dataset.GetRasterBand(1).WriteArray(im_data.squeeze())
    else:
        for i in range(im_bands):
            new_dataset.GetRasterBand(i + 1).WriteArray(im_data[i])

    del new_dataset

def read_img(sr_img):
    """read img

    Args:
        sr_img: The full path of the original image

    """
    im_dataset = gdal.Open(sr_img)
    if im_dataset == None:
        print('open sr_img false')
        sys.exit(1)
    im_geotrans = im_dataset.GetGeoTransform()
    im_proj = im_dataset.GetProjection()
    im_width = im_dataset.RasterXSize
    im_height = im_dataset.RasterYSize
    im_bands = im_dataset.RasterCount 
    im_data = im_dataset.ReadAsArray(0, 0, im_width, im_height)
    del im_dataset

    return im_data, im_proj, im_geotrans, im_height, im_width, im_bands

In [12]:
ref_img_path = r'D:\xj_parcel_2023\output\manas_ori_rgb_stretch.tif'
target_img_path = r'd:\xj_parcel_2023\output\manas_UNet_farmland_sobel_Gradient Magnitude.png'

save_img_path = target_img_path[:-4] + '_add_coord.tif'

ref_data, ref_proj, ref_geotrans = read_img(ref_img_path)[:3]

im_data, = read_img(target_img_path)[:1]

write_img(save_img_path, ref_proj, ref_geotrans, im_data)
